### Ingestion del archivo "language.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")


#### Paso 1 - Leer el archivo CSV utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
language_schema = StructType(fields =[

    StructField("languageid", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])

In [0]:
language_df = spark.read \
    .option("header",True) \
    .schema(language_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

#### Paso 2 - Seleccionar sólo las columnas "requeridas"

In [0]:
from pyspark.sql.functions import col

In [0]:
language_selected_df = language_df.select(col("languageId"), col("languageName"))

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
language_renamed_df = language_selected_df \
    .withColumnRenamed("languageId","language_id") \
    .withColumnRenamed("languageName","language_name")

#### Paso 4 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
language_final_df = add_ingestion_date(language_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))

#### Paso 5 - Escribir datos en Datalake en formato parquet

In [0]:
#language_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/language")
#language_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.language");
language_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language");

In [0]:
%sql
SELECT * FROM movie_silver.language;